# Iceberg on object stores

NOTE: THIS NOTEBOOK IS WORK IN PROGRESS.

In the Parquet module, we saw how query engines make byte-range requests to object stores to read only the relevant chunks of a Parquet file. Iceberg adds another layer: before touching any data file, it reads a chain of small metadata files.

In this notebook:

* Why object stores are the standard for data lakes
* How to configure Iceberg with S3
* What requests Iceberg makes to S3 to serve a query
* The small files problem and compaction
* Object store-specific design considerations

We'll use the same simulated S3 setup from the Parquet module so you can observe exactly which HTTP requests Iceberg makes.

## Object stores for data lakes

Object stores (AWS S3, Google Cloud Storage, Azure Blob Storage) are the dominant storage layer for modern data lakes. Unlike local filesystems:

| Property | Local filesystem | Object store |
|----------|-----------------|--------------|
| Latency | < 1 ms | 50-200 ms |
| Throughput | High | Very high (parallel requests) |
| Cost | Expensive | Very cheap |
| Durability | Single disk | 99.999999999% |
| Consistency | Immediate | Eventual (S3: strong after 2020) |
| Scalability | Limited | Virtually unlimited |
| Operations | POSIX | GET, PUT, DELETE, LIST |

Object stores have no true directories, no random writes, and no append operations - only full object reads and writes. This means:

* **No in-place updates**: To update a file, you must write a new one
* **No atomic multi-file operations**: You must coordinate this yourself (Iceberg does this via the catalog)
* **LIST is expensive**: Listing millions of objects is slow and costly

Iceberg is specifically designed for these constraints. Its metadata layer:
* Enables **atomic commits** without filesystem atomicity (via catalog)
* Eliminates **LIST calls** (manifests track all files explicitly)
* Minimizes **requests** (metadata chain tells you exactly which files to read)

In [ ]:
import os
import sys
import shutil
import daft
import pyarrow as pa
import boto3
from pathlib import Path
from pyiceberg.catalog.sql import SqlCatalog
from datetime import datetime

# Import the S3Simulator from the Parquet module
sys.path.insert(0, '../01_parquet')
from s3simulator import S3Simulator

os.environ["DAFT_DASHBOARD_ENABLED"] = "0"
os.environ["DAFT_PROGRESS_BAR"] = "0"

BUCKET = "iceberg-lake"
PORT = 5001  # Use different port than Parquet module to avoid conflicts
ENDPOINT = f"http://127.0.0.1:{PORT}"

# Start mock S3 server
s3_sim = S3Simulator(bucket_name=BUCKET, port=PORT)
s3_sim.start()

# Create S3 client for inspection
s3_client = boto3.client(
    's3',
    endpoint_url=ENDPOINT,
    aws_access_key_id='fake',
    aws_secret_access_key='fake',
    region_name='us-east-1'
)

print(f"S3 simulator running at {ENDPOINT}")
print(f"Bucket: {BUCKET}")

## Configuring Iceberg for S3

To store an Iceberg table on S3, we need two things:

1. **A catalog**: We'll keep using SQLite (stored locally) to manage table metadata pointers. In production this would be a REST Catalog, AWS Glue, or similar.
2. **S3 as the warehouse**: All table data and metadata files (JSON, AVRO, Parquet) go to S3.

The catalog configuration tells PyIceberg:
* Where to find S3 (`s3.endpoint`)
* How to authenticate (`s3.access-key-id`, `s3.secret-access-key`)
* Where to write data (`warehouse` pointing to `s3://...`)

This is the same configuration that would work against real AWS S3 - just replace the endpoint URL and credentials.

In [ ]:
# SQLite catalog stored locally (just the pointer to metadata)
catalog_dir = Path('../data/catalog_s3_demo').absolute()
catalog_dir.mkdir(parents=True, exist_ok=True)
catalog_db = catalog_dir / 'catalog.db'
catalog_db.unlink(missing_ok=True)

# Create Iceberg catalog with S3 warehouse
catalog = SqlCatalog(
    's3_demo',
    **{
        'uri': f'sqlite:///{catalog_db}',
        'warehouse': f's3://{BUCKET}/warehouse',
        's3.endpoint': ENDPOINT,
        's3.access-key-id': 'fake',
        's3.secret-access-key': 'fake',
        's3.region': 'us-east-1',
        's3.path-style-access': 'true',
    }
)
catalog.create_namespace('iot')

print("✅ Catalog initialized")
print(f"   Catalog DB (local): {catalog_db}")
print(f"   Warehouse (S3): s3://{BUCKET}/warehouse")

## Writing an Iceberg table to S3

Writing to S3 works exactly like writing to local files - we use the same PyIceberg API. The only difference is that all file I/O goes through S3. Watch the S3 simulator output above to see the PUT requests as each file is written.

In [ ]:
# Load events data
df_events = daft.read_json('../data/input/events.jsonl')
df_batch = df_events.limit(30000)
arrow_batch = df_batch.to_arrow()

# Create table and write first batch
print("Creating table and writing first batch (watch the PUT requests above)...")
print("=" * 60)

events_table = catalog.create_table(
    'iot.events',
    schema=pa.schema(arrow_batch.schema)
)
events_table.append(arrow_batch)

print("=" * 60)
print(f"\n✅ Appended {len(arrow_batch):,} records")

### What got written to S3?

Let's see exactly which objects Iceberg created in S3. You'll recognize the metadata hierarchy from the earlier notebook:

```
warehouse/
  iot/
    events/
      metadata/   ← JSON and AVRO metadata files
      data/       ← Parquet data files
```

In [ ]:
def list_s3_objects(prefix=''):
    """List all objects in the S3 bucket with their sizes."""
    paginator = s3_client.get_paginator('list_objects_v2')
    objects = []
    for page in paginator.paginate(Bucket=BUCKET, Prefix=prefix):
        for obj in page.get('Contents', []):
            objects.append(obj)
    return objects

objects = list_s3_objects('warehouse/')

print(f"Objects in S3 bucket ({len(objects)} total):")
print()

total_size = 0
metadata_size = 0
data_size = 0

for obj in sorted(objects, key=lambda x: x['Key']):
    key = obj['Key'].replace(f'warehouse/iot/events/', '')
    size = obj['Size']
    total_size += size

    # Categorize by file type
    if '.metadata.json' in obj['Key']:
        category = '📄 Metadata JSON'
        metadata_size += size
    elif '.avro' in obj['Key']:
        category = '📋 AVRO (manifest)'
        metadata_size += size
    elif '.parquet' in obj['Key']:
        category = '💾 Parquet (data)'
        data_size += size
    else:
        category = '📁 Other'

    print(f"  {category:25s} {size:>10,} bytes  {key}")

print()
print(f"Metadata files: {metadata_size / 1024:.1f} KB")
print(f"Data files:     {data_size / 1024 / 1024:.2f} MB")
print(f"Overhead ratio: {metadata_size / (data_size + 1) * 100:.2f}%")

In [ ]:
# Write a second batch to create another snapshot
print("Writing second batch (watch more PUT requests)...")
print("=" * 60)

df_batch2 = df_events.offset(30000).limit(20000)
arrow_batch2 = df_batch2.to_arrow()
events_table.append(arrow_batch2)

print("=" * 60)

# List again to see what was added
objects2 = list_s3_objects('warehouse/')
new_objects = [o for o in objects2 if o not in objects]

print(f"\n✅ Appended {len(arrow_batch2):,} more records")
print(f"New objects created: {len(objects2) - len(objects)}")
for obj in sorted(objects2, key=lambda x: x['Key']):
    if obj not in objects:
        key = obj['Key'].replace(f'warehouse/iot/events/', '')
        print(f"  + {obj['Size']:>10,} bytes  {key}")

## Reading from S3

Reading an Iceberg table from S3 follows the same metadata chain we traced in the metadata deep-dive notebook:

1. **Catalog lookup** (local SQLite) → current metadata JSON path in S3
2. **GET metadata JSON** from S3 → current snapshot → manifest list path
3. **GET manifest list** from S3 → list of manifest paths
4. **GET manifests** from S3 → list of data file paths + statistics
5. **GET data files** from S3 → byte-range requests for relevant row groups

For Daft to read Iceberg on S3, we need to provide S3 credentials in the IO config, just like we did in the Parquet module.

In [ ]:
io_config = daft.io.IOConfig(
    s3=daft.io.S3Config(
        endpoint_url=ENDPOINT,
        key_id='fake',
        access_key='fake',
        region_name='us-east-1',
    )
)

print("Reading Iceberg table from S3 (watch GET requests above)...")
print("=" * 60)

df = daft.read_iceberg(events_table, io_config=io_config)
result = daft.sql("SELECT COUNT(*) as total FROM df").collect()

print("=" * 60)
print(f"\nTotal records: {result.to_pydict()['total'][0]:,}")

In [ ]:
# Run a filtered query - observe the sequence of requests
print("Running filtered query (type = 'c8y_LocationUpdate')...")
print("=" * 60)
print("Notice the request sequence:")
print("  1. GET metadata JSON (small, catalog tells us where)")
print("  2. GET manifest list AVRO (small)")
print("  3. GET manifest AVRO files (small, check statistics)")
print("  4. GET parquet data files (only files that pass filter)")
print()

df_filtered = daft.read_iceberg(events_table, io_config=io_config)
df_filtered.filter(daft.col('type') == 'c8y_LocationUpdate').select('id', 'type', 'time').show(5)

print("=" * 60)

## Understanding the request pattern

Look at the requests logged above. You'll see:

* **Metadata requests** (`GET /warehouse/iot/events/metadata/...`): Always small, fast. These are the JSON and AVRO files.
* **Data requests** (`GET /warehouse/iot/events/data/...`): These are byte-range requests to Parquet files.

The key insight: **Iceberg reads metadata first, then only the relevant data**. For a query that touches 1% of data, Iceberg reads:
* 100% of metadata files (small, fast)
* ~1% of data files (potentially with byte-range precision)

### Why this matters for object stores

Every S3 GET request has fixed overhead (~50ms latency). Minimizing the number of requests is critical. Iceberg's approach:

1. **No directory listing**: Manifests track all files - no expensive S3 LIST calls
2. **Hierarchical metadata**: Skip entire manifest files based on partition info before reading per-file statistics
3. **Statistics-based pruning**: Use min/max bounds to skip files without reading them
4. **Byte-range reads**: Within files, read only relevant columns and row groups

In [ ]:
# Demonstrate how the object count grows with operations
print("Tracking S3 object count across operations:\n")

def count_s3_objects():
    objects = list_s3_objects('warehouse/')
    metadata_count = sum(1 for o in objects if '/metadata/' in o['Key'])
    data_count = sum(1 for o in objects if '/data/' in o['Key'])
    return len(objects), metadata_count, data_count

total, meta, data = count_s3_objects()
print(f"After 2 appends: {total} total ({meta} metadata, {data} data)")

# Another append
df_batch3 = df_events.offset(50000).limit(10000)
events_table.append(df_batch3.to_arrow())
total, meta, data = count_s3_objects()
print(f"After 3 appends: {total} total ({meta} metadata, {data} data)")

# A delete operation
print("\nRunning a delete operation...")
events_table.delete("type = 'OperationMode'")
total, meta, data = count_s3_objects()
print(f"After delete:    {total} total ({meta} metadata, {data} data)")

print("\nEach operation adds new files. Old files remain until cleaned up.")
print("This is Iceberg's append-only, immutable file model.")

## The small files problem

Object stores perform best with large files (hundreds of MB to GB). Small files cause:

* **High request overhead**: Each file needs one S3 GET request (~50-200ms). 10,000 small files = 10,000 requests.
* **Metadata bloat**: Each file gets an entry in the manifest. Large manifests slow down query planning.
* **Poor compression**: Small files miss compression opportunities that span row groups.

### How small files accumulate

* **Frequent small appends**: Each streaming micro-batch creates one or more small files
* **Many small partitions**: Fine-grained partitioning multiplied by many writers
* **Delete files**: Position delete files accumulate with frequent row-level deletes

### The solution: Compaction

**Compaction** (also called **file rewriting** or **optimize**) merges small files into larger ones. Iceberg supports compaction through Apache Spark's `OPTIMIZE` command and other engines. PyIceberg does not yet implement `rewrite_data_files()`, so we'll cover compaction hands-on in the Spark module.

## Production considerations

### Catalog choice for S3

In production, you wouldn't use SQLite as the catalog. The catalog must be:
* **Highly available**: Tables can't be accessed if catalog is down
* **Strongly consistent**: Concurrent writes must be serializable
* **Scalable**: Handle thousands of tables and millions of operations

Common production choices:

* **AWS Glue Data Catalog**: Native AWS service, works well with S3, integrates with Athena, EMR, Glue ETL
* **Lakekeeper (REST Catalog)**: Lightweight Rust implementation, runs as a service, works with any object store
* **Apache Polaris (REST Catalog)**: Java-based, cooperatively developed by Snowflake/Dremio
* **Project Nessie**: Git-like catalog with branching and multi-table transactions

### File size recommendations

| Data volume | Target file size | Row group size |
|-------------|-----------------|----------------|
| Small table (< 1 GB) | 128 MB | 8 MB |
| Medium table (1-100 GB) | 256-512 MB | 16-32 MB |
| Large table (> 100 GB) | 512 MB - 1 GB | 32-64 MB |

### Snapshot expiration and orphan file deletion

After writes and deletes, old files accumulate in S3:

1. `expire_snapshots()` - removes old snapshot metadata (PyIceberg supports this)
2. Delete orphan files - removes unreferenced data files left behind by expired snapshots or failed commits

PyIceberg does not yet implement orphan file deletion. Apache Spark's `RemoveOrphanFiles` is the standard approach for this in production.

In [ ]:
from datetime import timedelta

# Demonstrate snapshot expiration (removes old snapshot metadata)
print("Snapshots before expiration:")
for snap in events_table.snapshots():
    ts = datetime.fromtimestamp(snap.timestamp_ms / 1000)
    print(f"  {snap.snapshot_id} @ {ts.strftime('%H:%M:%S')}")

# Expire all snapshots older than 2 seconds (for demonstration)
events_table.maintenance.expire_snapshots().older_than(
    datetime.now() - timedelta(seconds=2)
).commit()

# Reload and check
events_table = catalog.load_table('iot.events')
print(f"\nSnapshots after expiration: {len(events_table.snapshots())}")
for snap in events_table.snapshots():
    ts = datetime.fromtimestamp(snap.timestamp_ms / 1000)
    print(f"  {snap.snapshot_id} @ {ts.strftime('%H:%M:%S')} (current)")

print()
print("Note: Metadata files for expired snapshots are removed from S3.")
print("Data files remain until orphan file cleanup is run.")

In [ ]:
# Final state of the S3 bucket
all_objects = list_s3_objects('warehouse/')

print(f"Final S3 state: {len(all_objects)} objects\n")

categories = {
    'Metadata JSON': [o for o in all_objects if '.metadata.json' in o['Key']],
    'Manifest list (AVRO)': [o for o in all_objects if 'snap-' in o['Key'] and '.avro' in o['Key']],
    'Manifest (AVRO)': [o for o in all_objects if '-m' in o['Key'] and '.avro' in o['Key']],
    'Data files (Parquet)': [o for o in all_objects if '.parquet' in o['Key']],
}

for category, objects in categories.items():
    total = sum(o['Size'] for o in objects)
    print(f"  {category}: {len(objects)} file(s), {total / 1024:.1f} KB")

In [ ]:
# Clean up
s3_sim.stop()

# Also clean up local catalog DB
shutil.rmtree(catalog_dir, ignore_errors=True)

print("✅ S3 simulator stopped and local files cleaned up")

## Review questions

**Why does Iceberg avoid S3 LIST operations?**
   - What does LIST return? Why is it expensive?
   - How do manifests replace LIST calls?

**What is the request sequence for a simple `SELECT COUNT(*) FROM events` on S3?**
   - How many S3 GETs are needed before reading any Parquet data?
   - What would change if there were 100 manifest files?

**How does object-store-backed Iceberg differ from local-file-backed Iceberg?**
   - Does the PyIceberg API change?
   - What changes in terms of performance characteristics?

**What is the difference between snapshot expiration and compaction?**
   - Which affects data files? Which affects metadata?
   - Can you run them independently?

**Why might you keep SQLite as the catalog even when using S3 as the warehouse?**
   - What does the catalog store vs. what goes in S3?
   - What are the limitations of SQLite in production?

**What happens to old data files after snapshot expiration?**
   - Are they deleted immediately?
   - What would happen to time-travel queries against expired snapshots?

## Challenges

### Trace a query's S3 requests

1. Restart the S3 simulator and create a fresh Iceberg table with 3 snapshots
2. Run `daft.read_iceberg(table).filter(daft.col('type') == 'c8y_Event').collect()`
3. Note down every S3 request in order (from the simulator log)
4. Classify each request: catalog / metadata JSON / manifest list / manifest / data file
5. How many requests are metadata vs. data?

### Measure the small files impact

1. Create two versions of the same table:
   * Version A: 50 writes of 1,000 records each (50 data files)
   * Version B: 1 write of 50,000 records (1 data file)
2. Run the same query on both
3. Count how many S3 requests are made for each version
4. Calculate the estimated latency difference (assuming 50ms per request)

### Compare catalog backends conceptually

Research and compare the following Iceberg catalogs for a hypothetical IoT production deployment:

* **AWS Glue** vs. **Lakekeeper** vs. **Project Nessie**

For each, evaluate:
1. How are concurrent writes handled?
2. What infrastructure is required?
3. Does it support multiple engines (Daft, Spark, Trino)?
4. What are the cost implications?

## Summary

Object stores are the natural home for Iceberg data lakes:

* **Designed for S3**: Iceberg's immutable files and append-only metadata work perfectly with object store semantics
* **No directory listings**: Manifests eliminate expensive S3 LIST operations
* **Minimal requests**: Metadata chain precisely identifies which data to read
* **Catalog is separate**: Catalog provides atomic commits; warehouse just stores files
* **Small files accumulate**: Frequent appends and deletes create many small files over time

### Request pattern for a query

```
Catalog (local) → Metadata JSON (1 GET)
                → Manifest list (1 GET)
                → Manifest files (n GETs, n = manifests that pass partition pruning)
                → Data files (m GETs, m = files that pass statistics pruning)
                   → Byte-range reads (per column chunk per row group)
```

### Key takeaways

1. **Object stores = no POSIX**: No appends, no atomic multi-file ops - Iceberg handles this
2. **Latency matters**: Each S3 request costs ~50-200ms; minimize request count
3. **Small files are expensive**: Aim for 128MB-1GB per Parquet file
4. **Compaction is essential but engine-dependent**: PyIceberg doesn't support it yet; use Spark
5. **Choose the right catalog**: SQLite for dev, REST Catalog for production
6. **Metadata overhead is tiny**: Typically < 1% of data size

### What's next?

You've now covered the complete Iceberg feature set with PyIceberg and Daft:

* **Getting started**: Creating and querying tables
* **Metadata deep dive**: How files link together
* **Time travel**: Querying historical snapshots
* **Schema evolution**: Changing schemas without rewrites
* **Transactions**: Optimistic concurrency control
* **Partitioning**: Efficient data organization for large scale
* **Object stores**: Production deployment on S3

The next module covers the **medallion architecture**: how to organize data lakes with bronze, silver, and gold layers using Iceberg and Parquet.